In [1]:
import os 
import numpy as np
import torch
import torch.nn as nn
import pandas as pd


In [2]:
def structure_file(df):
    df = df[0].str.split('\t', n=4, expand=True)

    df.columns = ["time", "sensor", "x", "y", "z"]

    GYR_data = df.loc[df['sensor'] == 'GYR']
    ACC_data = df.loc[df['sensor'] == 'ACC']

    GYR_data.index = GYR_data['time'].apply(pd.to_numeric)
    ACC_data.index = ACC_data['time'].apply(pd.to_numeric)
    GYR_data = GYR_data.drop('sensor', axis=1)
    ACC_data = ACC_data.drop('sensor', axis=1)
    GYR_data.columns = ['time_gyr', "gyr_x", "gyr_y", "gyr_z"]
    ACC_data.columns = ['time_acc', "acc_x", "acc_y", "acc_z"]
    
    GYR_data = GYR_data.apply(pd.to_numeric)
    ACC_data = ACC_data.apply(pd.to_numeric)
    
    min_idx = np.argmin([ACC_data.shape[0], GYR_data.shape[0]]) 
    max_idx = np.argmax([ACC_data.shape[0], GYR_data.shape[0]])  

    data_acc_gyr = [ACC_data, GYR_data] 
    data_acc_gyr[max_idx] = data_acc_gyr[max_idx].reindex(data_acc_gyr[min_idx].index, method='nearest', tolerance=0.02)
    new_set = pd.concat(data_acc_gyr, axis=1)
    new_set2 = new_set.dropna().reset_index(drop=True)
    return new_set2

In [7]:
data_running_oscar = structure_file(pd.read_csv('running/running_oscar.txt', header=None))
data_walking_walking = structure_file(pd.read_csv('walking/walking_oscar.txt', header=None))
data_standing_oscar = structure_file(pd.read_csv('standing_still/standing_still_oscar.txt', header=None))

data_running_oscar = structure_file(pd.read_csv('running/running_oscar.txt', header=None))
data_walking_walking = structure_file(pd.read_csv('walking/walking_oscar.txt', header=None))
data_standing_oscar = structure_file(pd.read_csv('standing_still/standing_still_oscar.txt', header=None))

all_data = {'running':[],
           'walking':[],
           'standing_still':[]}


data_types = ["standing_still", "walking", "running"]
current_path = os.path.abspath('.')
for data_type in data_types:
    data_path = os.path.join(current_path, data_type)
    for file in os.listdir(data_path):
        file_path = os.path.join(data_path, file)
        data = pd.read_csv(file_path, header=None)
        pandas_data = structure_file(data)
        all_data[data_type].append(pandas_data)
        # pprint(pandas_data)


1
2
3
4
5
6
7
8
9


In [4]:
# Create data sets
from sklearn.model_selection import train_test_split
# create sequence of data
num_channels = 6
time_series_len = 200

num_samples = 0
for key in all_data:
    for data_sample in all_data[key]:
        nr_sample = 0
        while nr_sample < data_sample.shape[0]//time_series_len -1:
            nr_sample += 1
            num_samples += 1            

data_np = np.zeros((num_samples, num_channels, time_series_len))
label_np = np.zeros(num_samples)

key_to_label = {'running': 0,
                'walking': 1,
                'standing_still': 2}

channels = ['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z']

nr_sample = 0
for key in all_data:
    nr_sub_sample = 0
    for data_sample in all_data[key]:
        start_v = 0
        while nr_sub_sample < data_sample.shape[0]//time_series_len - 1:
            label_np[nr_sample] = key_to_label[key] 
            for i in range(num_channels):
                data_key = channels[i]
                data_np[nr_sample, i, :] = data_sample[data_key].iloc[start_v: start_v+time_series_len]
            
            start_v += time_series_len
            nr_sub_sample += 1
            nr_sample += 1
            

X_train, X_test_val, y_train, y_test_val = train_test_split(data_np, label_np, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42)

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 2., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       2., 1., 2., 2., 1., 1., 0., 1., 0., 0., 2., 1., 0., 0., 0., 0., 0.,
       2., 1., 0., 0., 0., 0., 0., 0., 2., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 2., 2., 2., 2., 2., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2.,
       2., 0., 1., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 2., 2., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 2., 2.,
       0., 0., 0., 1., 1., 0., 1., 2., 0., 0., 0., 0., 2., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 1., 1., 2., 2.,
       1., 0., 0., 0., 1., 0., 2., 0., 1., 0., 2., 0., 0., 1., 0., 2., 0.,
       0., 0., 1., 2., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0., 0., 0., 2.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 2., 0., 0., 0.,
       0., 0., 1., 0., 0.